# TP2: Clasificación de Fashion-MNIST con PyTorch

## Trabajo Práctico 2 - Redes Neuronales 2025

### Referencias:
- https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
- https://github.com/zalandoresearch/fashion-mnist
- https://github.com/pranay414/Fashion-MNIST-Pytorch/blob/master/fashion_mnist.ipynb



---

## Sección 1: Importación de Librerías



In [ ]:
# Librerías estándar de Python
import os
import pickle
import datetime
from collections import defaultdict



In [ ]:
# Librerías third party de Python
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy as sp
import scipy.linalg as linalg
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import seaborn as sns
import json



In [ ]:
# Librerías de PyTorch
import torch
import torch.optim as optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Compose

# Crear directorio de imágenes si no existe
os.makedirs('images', exist_ok=True)

# Función helper para escribir resultados en markdown
results_file = 'resultados_experimentos.md'

def init_results_file():
    """Inicializa el archivo de resultados"""
    with open(results_file, 'w', encoding='utf-8') as f:
        f.write('# Resultados de Experimentos - TP2 Fashion-MNIST\n\n')
        f.write(f'**Fecha de ejecución**: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n\n')
        f.write('---\n\n')
        f.write('## Tabla de Contenidos\n\n')
        f.write('1. [Configuración del Entorno](#configuración-del-entorno)\n')
        f.write('2. [Dataset Fashion-MNIST](#dataset-fashion-mnist)\n')
        f.write('3. [Entrenamiento Básico](#entrenamiento-básico)\n')
        f.write('4. [Análisis de Hiperparámetros](#análisis-de-hiperparámetros)\n')
        f.write('5. [Modelo Final](#modelo-final)\n\n')
        f.write('---\n\n')

def write_section(title, level=2):
    """Escribe una sección en el archivo de resultados"""
    with open(results_file, 'a', encoding='utf-8') as f:
        f.write(f'\n{"#" * level} {title}\n\n')

def write_text(text):
    """Escribe texto en el archivo de resultados"""
    with open(results_file, 'a', encoding='utf-8') as f:
        f.write(f'{text}\n')

def write_table(headers, rows):
    """Escribe una tabla en markdown"""
    with open(results_file, 'a', encoding='utf-8') as f:
        # Headers
        f.write('| ' + ' | '.join(headers) + ' |\n')
        f.write('| ' + ' | '.join(['---'] * len(headers)) + ' |\n')
        # Rows
        for row in rows:
            f.write('| ' + ' | '.join(str(cell) for cell in row) + ' |\n')
        f.write('\n')

def write_image(image_path, caption=None):
    """Escribe referencia a imagen en markdown"""
    with open(results_file, 'a', encoding='utf-8') as f:
        if caption:
            f.write(f'**{caption}**\n\n')
        f.write(f'![{caption or "Imagen"}]({image_path})\n\n')

# Inicializar archivo de resultados
init_results_file()
print(f'Archivo de resultados inicializado: {results_file}')



---

## Sección 2: Configuración del Dispositivo (CPU/GPU)



In [ ]:
# Detectar y configurar dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Dispositivo utilizado: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'CUDA Version: {torch.version.cuda}')

# Documentar configuración
write_section('Configuración del Entorno', 2)
write_text(f'- **Dispositivo**: {device}')
if torch.cuda.is_available():
    write_text(f'- **GPU**: {torch.cuda.get_device_name(0)}')
    write_text(f'- **CUDA Version**: {torch.version.cuda}')
write_text(f'- **PyTorch Version**: {torch.__version__}')
write_text('')



---

## Sección 3: Carga y Exploración del Dataset Fashion-MNIST

### 3.1: Descargar y cargar el dataset



In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normaliza de [0,1] a [-1,1]
])

# Download and load the training data
train_set = datasets.FashionMNIST('MNIST_data/', download=True, train=True, transform=transform)
valid_set = datasets.FashionMNIST('MNIST_data/', download=True, train=False, transform=transform)



### 3.2: Explorar el dataset



In [ ]:
# Inspeccionar tamaño de los conjuntos
print(f'Tamaño del conjunto de entrenamiento: {len(train_set)}')
print(f'Tamaño del conjunto de validación: {len(valid_set)}')

# Ver un ejemplo
image, label = train_set[0]
print(f'\nDimensiones de la imagen: {image.shape}')
print(f'Etiqueta: {label}')
print(f'Tipo de datos: {type(image)}')

# Documentar dataset
write_section('Dataset Fashion-MNIST', 2)
write_text(f'- **Tamaño conjunto de entrenamiento**: {len(train_set)}')
write_text(f'- **Tamaño conjunto de validación**: {len(valid_set)}')
write_text(f'- **Dimensiones de imagen**: {image.shape}')
write_text(f'- **Número de clases**: 10')
write_text('')



In [ ]:
# Diccionario de nombres de clases (según paper original)
class_names = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boots'
}

print('Clases de Fashion-MNIST:')
for idx, name in class_names.items():
    print(f'  {idx}: {name}')



### 3.3: Visualización inicial - Mosaico de imágenes



In [ ]:
# Crear mosaico 3x3 de imágenes aleatorias
figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 3

for i in range(1, cols * rows + 1):
    j = torch.randint(len(train_set), size=(1,)).item()
    image, label = train_set[j]
    
    figure.add_subplot(rows, cols, i)
    plt.title(f'{class_names[label]} ({label})')
    plt.axis("off")
    plt.imshow(image.squeeze(), cmap="Greys_r")

plt.tight_layout()
plt.savefig('images/fashion_mnist_ejemplos.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar visualización
write_image('images/fashion_mnist_ejemplos.png', 'Mosaico 3x3 de ejemplos del dataset Fashion-MNIST')



---

## Sección 4: Definición de la Arquitectura de la Red Neuronal

### 4.1: Crear DataLoaders



In [ ]:
# Crear DataLoaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=100, shuffle=True)

# Explorar un batch
images, labels = next(iter(train_loader))
print(f'Dimensiones del batch de imágenes: {images.shape}')
print(f'Dimensiones del batch de etiquetas: {labels.shape}')



### 4.2: Definir la arquitectura de la red

Arquitectura:
- Capa de entrada: Flatten de 28×28 = 784 neuronas
- Capa oculta 1: 128 neuronas + ReLU + Dropout(0.2)
- Capa oculta 2: 64 neuronas + ReLU + Dropout(0.2)
- Capa de salida: 10 neuronas (sin activación, CrossEntropyLoss aplica softmax)



In [ ]:
# Definir la red neuronal
class FashionMNIST_Net(nn.Module):
    def __init__(self):
        super(FashionMNIST_Net, self).__init__()
        # Capa de entrada: 28x28 = 784
        self.flatten = nn.Flatten()
        
        # Capas ocultas
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        
        # Capa de salida
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x



### 4.3: Verificar la arquitectura



In [ ]:
# Crear instancia del modelo y verificar
model = FashionMNIST_Net()
model.to(device)

# Probar con un batch de ejemplo
test_images, _ = next(iter(train_loader))
test_images = test_images.to(device)
output = model(test_images)

print(f'Dimensiones de entrada: {test_images.shape}')
print(f'Dimensiones de salida: {output.shape}')
print(f'Modelo en dispositivo: {next(model.parameters()).device}')



---

## Sección 5: Funciones de Entrenamiento y Validación



In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Entrena el modelo por una época.
    
    Returns:
        train_loss: pérdida promedio de la época
        train_acc: precisión promedio de la época
    """
    model.train()  # Modo entrenamiento
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        # Mover datos al dispositivo
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calcular métricas
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    
    return train_loss, train_acc



In [ ]:
def validate(model, valid_loader, criterion, device):
    """
    Evalúa el modelo sobre el conjunto de validación.
    
    Returns:
        val_loss: pérdida promedio
        val_acc: precisión promedio
    """
    model.eval()  # Modo evaluación
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in valid_loader:
            # Mover datos al dispositivo
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Calcular métricas
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = running_loss / len(valid_loader)
    val_acc = 100 * correct / total
    
    return val_loss, val_acc



---

## Sección 6: Entrenamiento Básico

### 6.1: Configurar entrenamiento



In [ ]:
# Función de pérdida
criterion = nn.CrossEntropyLoss()

# Reinicializar modelo
model = FashionMNIST_Net()
model.to(device)

# Optimizador (debe crearse DESPUÉS de reinicializar el modelo)
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(f'Modelo configurado en: {device}')
print(f'Learning rate: {learning_rate}')
print(f'Optimizador: SGD')



### 6.2: Loop principal de entrenamiento



In [ ]:
# Inicializar listas para guardar métricas
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Número de épocas
num_epochs = 30

print(f'Iniciando entrenamiento por {num_epochs} épocas...')
print('-' * 60)

for epoch in range(num_epochs):
    # Entrenar
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validar
    val_loss, val_acc = validate(model, valid_loader, criterion, device)
    
    # Guardar métricas
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # Imprimir progreso
    print(f'Época [{epoch+1}/{num_epochs}]')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print('-' * 60)

print('Entrenamiento completado!')

# Documentar entrenamiento básico
write_section('Entrenamiento Básico', 2)
write_text('### Configuración')
write_text(f'- **Optimizador**: SGD')
write_text(f'- **Learning Rate**: {learning_rate}')
write_text(f'- **Batch Size**: 100')
write_text(f'- **Épocas**: {num_epochs}')
write_text(f'- **Dropout**: 0.2')
write_text('')
write_text('### Resultados Finales')
write_text(f'- **Train Accuracy**: {train_accuracies[-1]:.2f}%')
write_text(f'- **Validation Accuracy**: {val_accuracies[-1]:.2f}%')
write_text(f'- **Train Loss**: {train_losses[-1]:.4f}')
write_text(f'- **Validation Loss**: {val_losses[-1]:.4f}')
write_text('')



---

## Sección 7: Visualización de Resultados Básicos

### 7.1: Gráficos de curvas de entrenamiento



In [ ]:
# Crear figura con curvas de entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de pérdida
ax1.plot(train_losses, label='Train Loss', marker='o')
ax1.plot(val_losses, label='Validation Loss', marker='s')
ax1.set_xlabel('Época')
ax1.set_ylabel('Cross Entropy Loss')
ax1.set_title('Pérdida durante el Entrenamiento')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico de precisión
ax2.plot(train_accuracies, label='Train Accuracy', marker='o')
ax2.plot(val_accuracies, label='Validation Accuracy', marker='s')
ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión (%)')
ax2.set_title('Precisión durante el Entrenamiento')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/curvas_entrenamiento_basico.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar visualización
write_image('images/curvas_entrenamiento_basico.png', 'Curvas de entrenamiento básico')



### 7.2: Matriz de confusión



In [ ]:
# Obtener todas las predicciones del conjunto de validación
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcular matriz de confusión
cm = confusion_matrix(all_labels, all_preds)

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[class_names[i] for i in range(10)],
            yticklabels=[class_names[i] for i in range(10)])
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión - Fashion-MNIST')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('images/matriz_confusion_basico.png', dpi=150, bbox_inches='tight')
plt.show()

# Calcular precisión general
accuracy = accuracy_score(all_labels, all_preds)
print(f'Precisión general en validación: {accuracy*100:.2f}%')

# Documentar matriz de confusión
write_image('images/matriz_confusion_basico.png', 'Matriz de confusión - Entrenamiento básico')
write_text(f'**Precisión general en validación**: {accuracy*100:.2f}%')
write_text('')



---

## Sección 8: Análisis de Hiperparámetros

Esta sección contiene experimentos para analizar el efecto de diferentes hiperparámetros en el rendimiento del modelo.


### 8.1: Variar Learning Rate

In [ ]:
# Probar diferentes learning rates
learning_rates = [0.0001, 0.001, 0.01]
results_lr = {}

for lr in learning_rates:
    print(f'\nEntrenando con learning rate = {lr}...')
    
    # Reinicializar modelo
    model = FashionMNIST_Net()
    model.to(device)
    
    # Configurar optimizador
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    # Listas para métricas
    train_losses_lr = []
    val_losses_lr = []
    train_accs_lr = []
    val_accs_lr = []
    
    # Entrenar por 10 épocas
    for epoch in range(10):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, valid_loader, criterion, device)
        
        train_losses_lr.append(train_loss)
        val_losses_lr.append(val_loss)
        train_accs_lr.append(train_acc)
        val_accs_lr.append(val_acc)
    
    results_lr[lr] = {
        'train_losses': train_losses_lr,
        'val_losses': val_losses_lr,
        'train_accs': train_accs_lr,
        'val_accs': val_accs_lr
    }
    
    print(f'Final - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

# Visualizar comparación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for lr in learning_rates:
    ax1.plot(results_lr[lr]['val_losses'], label=f'LR={lr}', marker='o')
    ax2.plot(results_lr[lr]['val_accs'], label=f'LR={lr}', marker='o')

ax1.set_xlabel('Época')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Comparación de Learning Rates - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Validation Accuracy (%)')
ax2.set_title('Comparación de Learning Rates - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/comparacion_learning_rates.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar experimento
write_section('Análisis de Hiperparámetros', 2)
write_section('Experimento 1: Variar Learning Rate', 3)
write_text('**Valores probados**: 0.0001, 0.001, 0.01')
write_text('**Configuración**: SGD, Batch Size=100, Épocas=10, Dropout=0.2')
write_text('')
headers = ['Learning Rate', 'Train Accuracy', 'Validation Accuracy']
rows = [[lr, f'{results_lr[lr]["train_accs"][-1]:.2f}%', f'{results_lr[lr]["val_accs"][-1]:.2f}%'] for lr in learning_rates]
write_table(headers, rows)
write_image('images/comparacion_learning_rates.png', 'Comparación de Learning Rates')
write_text('')



### 8.2: Comparar Optimizadores (SGD vs ADAM)



In [ ]:
# Comparar SGD vs ADAM
optimizers_config = {
    'SGD': torch.optim.SGD,
    'ADAM': torch.optim.Adam
}

results_opt = {}

for opt_name, opt_class in optimizers_config.items():
    print(f'\nEntrenando con {opt_name}...')
    
    # Reinicializar modelo
    model = FashionMNIST_Net()
    model.to(device)
    
    # Configurar optimizador
    optimizer = opt_class(model.parameters(), lr=0.001)
    
    # Listas para métricas
    train_losses_opt = []
    val_losses_opt = []
    train_accs_opt = []
    val_accs_opt = []
    
    # Entrenar por 10 épocas
    for epoch in range(10):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, valid_loader, criterion, device)
        
        train_losses_opt.append(train_loss)
        val_losses_opt.append(val_loss)
        train_accs_opt.append(train_acc)
        val_accs_opt.append(val_acc)
    
    results_opt[opt_name] = {
        'train_losses': train_losses_opt,
        'val_losses': val_losses_opt,
        'train_accs': train_accs_opt,
        'val_accs': val_accs_opt
    }
    
    print(f'Final - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

# Visualizar comparación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for opt_name in optimizers_config.keys():
    ax1.plot(results_opt[opt_name]['val_losses'], label=opt_name, marker='o')
    ax2.plot(results_opt[opt_name]['val_accs'], label=opt_name, marker='o')

ax1.set_xlabel('Época')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Comparación de Optimizadores - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Validation Accuracy (%)')
ax2.set_title('Comparación de Optimizadores - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/comparacion_optimizadores.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar experimento
write_section('Experimento 2: Comparar Optimizadores (SGD vs ADAM)', 3)
write_text('**Optimizadores probados**: SGD, ADAM')
write_text('**Configuración**: LR=0.001, Batch Size=100, Épocas=10, Dropout=0.2')
write_text('')
headers = ['Optimizador', 'Train Accuracy', 'Validation Accuracy']
rows = [[opt, f'{results_opt[opt]["train_accs"][-1]:.2f}%', f'{results_opt[opt]["val_accs"][-1]:.2f}%'] for opt in optimizers_config.keys()]
write_table(headers, rows)
write_image('images/comparacion_optimizadores.png', 'Comparación de Optimizadores')
write_text('')



### 8.3: Variar valor de Dropout


In [ ]:
# Definir una función para crear modelos con diferentes valores de dropout
def create_model_with_dropout(dropout_rate):
    """Crea un modelo con un valor específico de dropout"""
    class FashionMNIST_Net_Dropout(nn.Module):
        def __init__(self, dropout_rate=0.2):
            super(FashionMNIST_Net_Dropout, self).__init__()
            self.flatten = nn.Flatten()
            self.fc1 = nn.Linear(784, 128)
            self.relu1 = nn.ReLU()
            self.dropout1 = nn.Dropout(dropout_rate)
            self.fc2 = nn.Linear(128, 64)
            self.relu2 = nn.ReLU()
            self.dropout2 = nn.Dropout(dropout_rate)
            self.fc3 = nn.Linear(64, 10)
        
        def forward(self, x):
            x = self.flatten(x)
            x = self.fc1(x)
            x = self.relu1(x)
            x = self.dropout1(x)
            x = self.fc2(x)
            x = self.relu2(x)
            x = self.dropout2(x)
            x = self.fc3(x)
            return x
    
    return FashionMNIST_Net_Dropout(dropout_rate)

# Probar diferentes valores de dropout
dropout_values = [0.0, 0.2, 0.4, 0.6]
results_dropout = {}

for dropout_rate in dropout_values:
    print(f'\nEntrenando con dropout = {dropout_rate}...')
    
    # Crear modelo con el valor de dropout específico
    model = create_model_with_dropout(dropout_rate)
    model.to(device)
    
    # Configurar optimizador
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    # Listas para métricas
    train_losses_dropout = []
    val_losses_dropout = []
    train_accs_dropout = []
    val_accs_dropout = []
    
    # Entrenar por 10 épocas
    for epoch in range(10):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, valid_loader, criterion, device)
        
        train_losses_dropout.append(train_loss)
        val_losses_dropout.append(val_loss)
        train_accs_dropout.append(train_acc)
        val_accs_dropout.append(val_acc)
    
    results_dropout[dropout_rate] = {
        'train_losses': train_losses_dropout,
        'val_losses': val_losses_dropout,
        'train_accs': train_accs_dropout,
        'val_accs': val_accs_dropout
    }
    
    print(f'Final - Train Acc: {train_accs_dropout[-1]:.2f}%, Val Acc: {val_accs_dropout[-1]:.2f}%')

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for dropout_rate in dropout_values:
    ax1.plot(results_dropout[dropout_rate]['val_losses'], 
             label=f'Dropout={dropout_rate}', marker='o')
    ax2.plot(results_dropout[dropout_rate]['val_accs'], 
             label=f'Dropout={dropout_rate}', marker='o')

ax1.set_xlabel('Época')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Comparación de Dropout - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Validation Accuracy (%)')
ax2.set_title('Comparación de Dropout - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/comparacion_dropout.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar experimento
write_section('Experimento 3: Variar Dropout', 3)
write_text('**Valores probados**: 0.0, 0.2, 0.4, 0.6')
write_text('**Configuración**: SGD, LR=0.001, Batch Size=100, Épocas=10')
write_text('')
headers = ['Dropout', 'Train Accuracy', 'Validation Accuracy']
rows = [[dr, f'{results_dropout[dr]["train_accs"][-1]:.2f}%', f'{results_dropout[dr]["val_accs"][-1]:.2f}%'] for dr in dropout_values]
write_table(headers, rows)
write_image('images/comparacion_dropout.png', 'Comparación de Dropout')
write_text('')


### 8.4: Variar número de neuronas en las capas intermedias


In [ ]:
# Definir función para crear modelos con diferentes arquitecturas
def create_model_with_neurons(n1, n2):
    """Crea un modelo con n1 y n2 neuronas en las capas ocultas"""
    class FashionMNIST_Net_Neurons(nn.Module):
        def __init__(self, n1=128, n2=64):
            super(FashionMNIST_Net_Neurons, self).__init__()
            self.flatten = nn.Flatten()
            self.fc1 = nn.Linear(784, n1)
            self.relu1 = nn.ReLU()
            self.dropout1 = nn.Dropout(0.2)
            self.fc2 = nn.Linear(n1, n2)
            self.relu2 = nn.ReLU()
            self.dropout2 = nn.Dropout(0.2)
            self.fc3 = nn.Linear(n2, 10)
        
        def forward(self, x):
            x = self.flatten(x)
            x = self.fc1(x)
            x = self.relu1(x)
            x = self.dropout1(x)
            x = self.fc2(x)
            x = self.relu2(x)
            x = self.dropout2(x)
            x = self.fc3(x)
            return x
    
    return FashionMNIST_Net_Neurons(n1, n2)

# Probar diferentes configuraciones de neuronas
neuron_configs = [
    (64, 32),   # Más pequeña
    (128, 64),  # Original
    (256, 128), # Más grande
    (512, 256)  # Mucho más grande
]

results_neurons = {}

for n1, n2 in neuron_configs:
    config_name = f'{n1}-{n2}'
    print(f'\nEntrenando con arquitectura {n1}-{n2}...')
    
    # Crear modelo
    model = create_model_with_neurons(n1, n2)
    model.to(device)
    
    # Configurar optimizador
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    # Listas para métricas
    train_losses_neurons = []
    val_losses_neurons = []
    train_accs_neurons = []
    val_accs_neurons = []
    
    # Entrenar por 10 épocas
    for epoch in range(10):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, valid_loader, criterion, device)
        
        train_losses_neurons.append(train_loss)
        val_losses_neurons.append(val_loss)
        train_accs_neurons.append(train_acc)
        val_accs_neurons.append(val_acc)
    
    results_neurons[config_name] = {
        'train_losses': train_losses_neurons,
        'val_losses': val_losses_neurons,
        'train_accs': train_accs_neurons,
        'val_accs': val_accs_neurons
    }
    
    print(f'Final - Train Acc: {train_accs_neurons[-1]:.2f}%, Val Acc: {val_accs_neurons[-1]:.2f}%')

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for config_name in results_neurons.keys():
    ax1.plot(results_neurons[config_name]['val_losses'], 
             label=f'{config_name}', marker='o')
    ax2.plot(results_neurons[config_name]['val_accs'], 
             label=f'{config_name}', marker='o')

ax1.set_xlabel('Época')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Comparación de Arquitecturas - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Validation Accuracy (%)')
ax2.set_title('Comparación de Arquitecturas - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/comparacion_neuronas.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar experimento
write_section('Experimento 4: Variar Número de Neuronas', 3)
write_text('**Configuraciones probadas**: (64,32), (128,64), (256,128), (512,256)')
write_text('**Configuración**: SGD, LR=0.001, Batch Size=100, Épocas=10, Dropout=0.2')
write_text('')
headers = ['Arquitectura', 'Train Accuracy', 'Validation Accuracy']
rows = [[config, f'{results_neurons[config]["train_accs"][-1]:.2f}%', f'{results_neurons[config]["val_accs"][-1]:.2f}%'] for config in results_neurons.keys()]
write_table(headers, rows)
write_image('images/comparacion_neuronas.png', 'Comparación de Arquitecturas')
write_text('')


### 8.5: Variar número de épocas de entrenamiento


In [ ]:
# Entrenar el mismo modelo con diferentes números de épocas
epochs_to_test = [5, 10, 15, 20, 30]
results_epochs = {}

for num_epochs in epochs_to_test:
    print(f'\nEntrenando por {num_epochs} épocas...')
    
    # Reinicializar modelo para cada experimento
    model = FashionMNIST_Net()
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    # Listas para métricas
    train_losses_epochs = []
    val_losses_epochs = []
    train_accs_epochs = []
    val_accs_epochs = []
    
    # Entrenar
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, valid_loader, criterion, device)
        
        train_losses_epochs.append(train_loss)
        val_losses_epochs.append(val_loss)
        train_accs_epochs.append(train_acc)
        val_accs_epochs.append(val_acc)
    
    results_epochs[num_epochs] = {
        'train_losses': train_losses_epochs,
        'val_losses': val_losses_epochs,
        'train_accs': train_accs_epochs,
        'val_accs': val_accs_epochs
    }
    
    print(f'Final - Train Acc: {train_accs_epochs[-1]:.2f}%, Val Acc: {val_accs_epochs[-1]:.2f}%')

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for num_epochs in epochs_to_test:
    epochs_list = list(range(1, num_epochs + 1))
    ax1.plot(epochs_list, results_epochs[num_epochs]['val_losses'], 
             label=f'{num_epochs} épocas', marker='o')
    ax2.plot(epochs_list, results_epochs[num_epochs]['val_accs'], 
             label=f'{num_epochs} épocas', marker='o')

ax1.set_xlabel('Época')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Comparación de Épocas - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Validation Accuracy (%)')
ax2.set_title('Comparación de Épocas - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/comparacion_epocas.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar experimento
write_section('Experimento 5: Variar Número de Épocas', 3)
write_text('**Valores probados**: 5, 10, 15, 20, 30')
write_text('**Configuración**: SGD, LR=0.001, Batch Size=100, Dropout=0.2')
write_text('')
headers = ['Épocas', 'Train Accuracy', 'Validation Accuracy']
rows = [[ep, f'{results_epochs[ep]["train_accs"][-1]:.2f}%', f'{results_epochs[ep]["val_accs"][-1]:.2f}%'] for ep in epochs_to_test]
write_table(headers, rows)
write_image('images/comparacion_epocas.png', 'Comparación de Épocas')
write_text('')


### 8.6: Variar tamaño de los lotes (batch size)


In [ ]:
# Probar diferentes tamaños de batch
batch_sizes = [32, 64, 100, 128, 256]
results_batch = {}

for batch_size in batch_sizes:
    print(f'\nEntrenando con batch_size = {batch_size}...')
    
    # Crear nuevos DataLoaders con el batch_size específico
    train_loader_batch = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True
    )
    valid_loader_batch = torch.utils.data.DataLoader(
        valid_set, batch_size=batch_size, shuffle=True
    )
    
    # Reinicializar modelo
    model = FashionMNIST_Net()
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    # Listas para métricas
    train_losses_batch = []
    val_losses_batch = []
    train_accs_batch = []
    val_accs_batch = []
    
    # Entrenar por 10 épocas
    for epoch in range(10):
        train_loss, train_acc = train_epoch(
            model, train_loader_batch, criterion, optimizer, device
        )
        val_loss, val_acc = validate(
            model, valid_loader_batch, criterion, device
        )
        
        train_losses_batch.append(train_loss)
        val_losses_batch.append(val_loss)
        train_accs_batch.append(train_acc)
        val_accs_batch.append(val_acc)
    
    results_batch[batch_size] = {
        'train_losses': train_losses_batch,
        'val_losses': val_losses_batch,
        'train_accs': train_accs_batch,
        'val_accs': val_accs_batch
    }
    
    print(f'Final - Train Acc: {train_accs_batch[-1]:.2f}%, Val Acc: {val_accs_batch[-1]:.2f}%')

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for batch_size in batch_sizes:
    ax1.plot(results_batch[batch_size]['val_losses'], 
             label=f'Batch={batch_size}', marker='o')
    ax2.plot(results_batch[batch_size]['val_accs'], 
             label=f'Batch={batch_size}', marker='o')

ax1.set_xlabel('Época')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Comparación de Batch Size - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Validation Accuracy (%)')
ax2.set_title('Comparación de Batch Size - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/comparacion_batch_size.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar experimento
write_section('Experimento 6: Variar Batch Size', 3)
write_text('**Valores probados**: 32, 64, 100, 128, 256')
write_text('**Configuración**: SGD, LR=0.001, Épocas=10, Dropout=0.2')
write_text('')
headers = ['Batch Size', 'Train Accuracy', 'Validation Accuracy']
rows = [[bs, f'{results_batch[bs]["train_accs"][-1]:.2f}%', f'{results_batch[bs]["val_accs"][-1]:.2f}%'] for bs in batch_sizes]
write_table(headers, rows)
write_image('images/comparacion_batch_size.png', 'Comparación de Batch Size')
write_text('')


---

## Sección 9: Generación de Figuras Finales

### 9.1: Entrenar modelo final con mejores hiperparámetros



In [ ]:
# Seleccionar mejores hiperparámetros basado en análisis previo
# Resultados del análisis:
# - ADAM: 88.06% validation accuracy (mejor que SGD: 73.44%)
# - Learning Rate 0.001: óptimo con ADAM
# - 30 épocas: mejor resultado (79.63%)
# - Batch size 32: mejor resultado (79.35%)

best_lr = 0.001  # Óptimo con ADAM
best_optimizer = 'ADAM'  # Mejor rendimiento: 88.06% vs 73.44% de SGD
optimal_epochs = 30  # Mejor resultado: 79.63% validation accuracy
optimal_batch_size = 32  # Mejor resultado: 79.35% validation accuracy

# Crear DataLoaders con batch size óptimo
train_loader_optimal = torch.utils.data.DataLoader(
    train_set, batch_size=optimal_batch_size, shuffle=True
)
valid_loader_optimal = torch.utils.data.DataLoader(
    valid_set, batch_size=optimal_batch_size, shuffle=True
)

# Reinicializar modelo
model_final = FashionMNIST_Net()
model_final.to(device)

# Configurar optimizador final
if best_optimizer == 'ADAM':
    optimizer_final = torch.optim.Adam(model_final.parameters(), lr=best_lr)
else:
    optimizer_final = torch.optim.SGD(model_final.parameters(), lr=best_lr)

# Listas para métricas finales
train_losses_final = []
train_accuracies_final = []
val_losses_final = []
val_accuracies_final = []

print(f'Entrenando modelo final con configuración óptima:')
print(f'  Optimizador: {best_optimizer}')
print(f'  Learning Rate: {best_lr}')
print(f'  Batch Size: {optimal_batch_size}')
print(f'  Épocas: {optimal_epochs}')
print('-' * 60)

for epoch in range(optimal_epochs):
    train_loss, train_acc = train_epoch(
        model_final, train_loader_optimal, criterion, optimizer_final, device
    )
    val_loss, val_acc = validate(
        model_final, valid_loader_optimal, criterion, device
    )
    
    train_losses_final.append(train_loss)
    train_accuracies_final.append(train_acc)
    val_losses_final.append(val_loss)
    val_accuracies_final.append(val_acc)
    
    if (epoch + 1) % 5 == 0:
        print(f'Época [{epoch+1}/{optimal_epochs}] - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

print('\nEntrenamiento final completado!')
print(f'Precisión final en validación: {val_accuracies_final[-1]:.2f}%')

# Documentar modelo final
write_section('Modelo Final', 2)
write_text('### Configuración Óptima Aplicada')
write_text(f'- **Optimizador**: {best_optimizer}')
write_text(f'- **Learning Rate**: {best_lr}')
write_text(f'- **Batch Size**: {optimal_batch_size}')
write_text(f'- **Épocas**: {optimal_epochs}')
write_text(f'- **Dropout**: 0.2')
write_text('')
write_text('### Resultados del Entrenamiento')
write_text('')
write_text('**Progreso por Épocas:**')
write_text('')
headers_epochs = ['Época', 'Train Accuracy', 'Validation Accuracy']
rows_epochs = []
for i in range(0, optimal_epochs, 5):
    if i < len(train_accuracies_final):
        rows_epochs.append([i+1, f'{train_accuracies_final[i]:.2f}%', f'{val_accuracies_final[i]:.2f}%'])
rows_epochs.append([optimal_epochs, f'{train_accuracies_final[-1]:.2f}%', f'{val_accuracies_final[-1]:.2f}%'])
write_table(headers_epochs, rows_epochs)
write_text('')
write_text(f'**Resultado Final:**')
write_text(f'- **Precisión en Validación**: {val_accuracies_final[-1]:.2f}%')
write_text(f'- **Precisión en Entrenamiento**: {train_accuracies_final[-1]:.2f}%')
write_text(f'- **Validation Loss**: {val_losses_final[-1]:.4f}')
write_text(f'- **Train Loss**: {train_losses_final[-1]:.4f}')
write_text('')

### 9.2: Generar figuras finales



In [ ]:
# Curvas de entrenamiento finales
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(train_losses_final, label='Train Loss', marker='o')
ax1.plot(val_losses_final, label='Validation Loss', marker='s')
ax1.set_xlabel('Época')
ax1.set_ylabel('Cross Entropy Loss')
ax1.set_title('Pérdida durante el Entrenamiento - Modelo Final')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(train_accuracies_final, label='Train Accuracy', marker='o')
ax2.plot(val_accuracies_final, label='Validation Accuracy', marker='s')
ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión (%)')
ax2.set_title('Precisión durante el Entrenamiento - Modelo Final')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/curvas_entrenamiento_final.png', dpi=150, bbox_inches='tight')
plt.show()

# Documentar visualización
write_image('images/curvas_entrenamiento_final.png', 'Curvas de entrenamiento - Modelo Final')
write_text('')



In [ ]:
# Matriz de confusión final
model_final.eval()
all_preds_final = []
all_labels_final = []

with torch.no_grad():
    for images, labels in valid_loader_optimal:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_final(images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds_final.extend(predicted.cpu().numpy())
        all_labels_final.extend(labels.cpu().numpy())

# Calcular matriz de confusión
cm_final = confusion_matrix(all_labels_final, all_preds_final)

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm_final, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[class_names[i] for i in range(10)],
            yticklabels=[class_names[i] for i in range(10)])
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión - Modelo Final')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('images/matriz_confusion_final.png', dpi=150, bbox_inches='tight')
plt.show()

# Calcular precisión final
accuracy_final = accuracy_score(all_labels_final, all_preds_final)
print(f'Precisión final en validación: {accuracy_final*100:.2f}%')

# Documentar matriz de confusión final
write_image('images/matriz_confusion_final.png', 'Matriz de confusión - Modelo Final')
write_text(f'**Precisión final en validación**: {accuracy_final*100:.2f}%')
write_text('')
write_text('---')
write_text('')
write_text(f'*Documento generado automáticamente el {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}*')
print(f'\n✅ Documento de resultados guardado en: {results_file}')

